In [11]:
# Wybierz dziedzinę STRIPS i trzy przykładowe problemy z conajmniej 50 stanami, np.stąd:https://github.com/primaryobjects/strips,gdzie rozwiązanie składa się z minimum 4 instancji akcji.Zdefiniuj je za pomocą STRIPS_domain i Planning_problem z AIPython:https://artint.info/AIPython/.

from aipython.stripsProblem import STRIPS_domain, Planning_problem

In [ ]:
starcraft_domain = STRIPS_domain(
  {
      #unit
    'unit': ['marine', 'medic', 'firebat', 'ghost', 'siege_tank', 'vulture', 'goliath', 'wraith', 'valkyrie', 'battlecruiser', 'scv', 'command_center', 'supply_depot', 'refinery', 'barracks', 'academy', 'factory', 'starport', 'control_tower', 'science_facility', 'physics_lab', 'engineering_bay', 'armory', 'missile_turret', 'bunker'],
      #location
    'location': ['base', 'barracks', 'factory', 'starport', 'mineral_field', 'vespene_geyser', 'enemy_base'],
        #resource
    'resource': ['mineral', 'vespene'],
        #upgrade
    'upgrade': ['infantry_weapons', 'infantry_armor', 'vehicle_weapons', 'vehicle_armor', 'ship_weapons', 'ship_armor'],
        #action
    'action': ['move', 'attack', 'build', 'harvest', 'train', 'research', 'upgrade']
  },
  {
      #actions
    'move': [('unit', 'location')],
    'attack': [('unit', 'unit')],
    'build': [('unit', 'unit')],
    'harvest': [('unit', 'resource')],
    'train': [('unit', 'unit')],
    'research': [('unit', 'upgrade')],
    'upgrade': [('unit', 'upgrade')]
  }
)

problem0 = Planning_problem(starcraft_domain,
                            {
                                'unit': 'marine',
                                'location': 'base',
                                'resource': 'mineral',
                                'upgrade': 'infantry_weapons'
                            },
                            {'location': 'enemy_base'}
)

def move(unit, location):
    return unit, location

def attack(unit, enemy):
    return unit, enemy

def build(unit, building):
    return unit, building

def harvest(unit, resource):
    return unit, resource

def train(unit, unit_type):
    return unit, unit_type

def research(unit, upgrade):
    return unit, upgrade

def upgrade(unit, upgrade):
    return unit, upgrade

def create_unit(unit, location, resource, upgrade):
    return unit, location, resource, upgrade

def create_building(building, location, resource, upgrade):
    return building, location, resource, upgrade



problem0.initial_state = {
    'unit': 'marine',
    'location': 'base',
    'resource': 'mineral',
    'upgrade': 'infantry_weapons'
}

problem0.goal_state = {
    'location': 'enemy_base'
}

problem0.actions = {
    'move': move,
    'attack': attack,
    'build': build,
    'harvest': harvest,
    'train': train,
    'research': research,
    'upgrade': upgrade
}

problem0.initial_state